API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [2]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangSmith 추적을 설정합니다.(선택)

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-EMBEDDING")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-EMBEDDING


In [3]:
# 라이브러리 선언
import sqlite3
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import re

In [5]:
# 1. 데이터베이스 연결 및 조회 (SQLite 일 경우)
sql = "SELECT * FROM PRODUCT"
# SQLite 데이터베이스 연결
conn = sqlite3.connect('../.db/sqlite/my_database.db')
cursor = conn.cursor()

# SQL 쿼리 실행
cursor.execute(sql)
rows = cursor.fetchall()

# 샘플 출력
for row in rows[:10]:
    print(row)

# 연결 종료
cursor.close()
conn.close()

('0021216860', '엔산 EL-09 이젤형 TV스탠드 티비거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '색상', '화이트 계열', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '176000', '176000', '0', '', '', '176000')
('0026595956', '엔산 무드조명형 모니터암 블룸스탠드 삼텐바이미 거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '', '', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '140000', '140000', '0', '', '', '140000')
('0037357623', '5년무상AS 이스트라 120Hz AI 맥스 127cm UHD 구글 5.0 스마트TV (이젤스탠드A타입-기사)', '정상상품', '이스트라', 'TV>UHD TV', '크기,종류,해상도,응답속도,기본 주사율,스마트기능,스마트기능,스마트기능,스마트기능,스마트기능,스마트기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,e효율등급,출시년도,화면 타입,스피커 출력,음향효과,음향효과,지원단자', '50인치(127cm),LED,UHD(4K),6m/s,120Hz,구글플레이,넷플릭스,유튜브,크롬캐스트,구글어시스턴트,Disney+,블루투스,게임모드,플리커프리,HDMI CEC,HDMI 2.1 지원,HDMI eARC,크로마서브샘플링(4:4:4),USB재생(동영상·사진·음악),1등급,2025년,평면형,24W,돌비디지털Plus,돌비디지털,HDMI', 'TV·영상가전>TV>중형(~139cm)', 'Y', '#AN507UHD07', '588000', '588000', '0', '', '', '546840')
('0021376941', '2024년형 50인치 스마트 UHD 구글 TV 4K 에너지효율 1등급 T5003KUG 스탠드직배송', '정상상품', '루컴즈', 'TV

In [5]:
# 2. 주어진 컬럼들을 바탕으로 ML 검색에 용이하도록 문맥에 맞게 병합하는 함수 정의

# 이 부분은 실제 데이터의 컬럼 순서와 일치해야 합니다.
COLUMN_INDICES = {
    "GOODS_NO": 0,
    "GOODS_NM": 1,
    "GOODS_STAT_SCT_NM": 2,
    "BRND_NM": 3,
    "ARTC_INFO": 4,
    "OPT_DISP_NM": 5,
    "OPT_VAL_DESC": 6,
    "CATEGORY_NMS": 7,
    "APPLIANCE_YN": 8,
    "SCH_KWD_NM": 9,
    "SALE_PRC": 10,
    "DSCNT_SALE_PRC": 11,
    "CARD_DC_RATE": 12,
    "CARD_DC_NAME": 13,
    "CARD_DC_NAME_LIST": 14,
    "MAX_BENEFIT_PRICE": 15
}

def generate_search_text_from_tuple(product_data_tuple):
    """
    주어진 상품 정보 튜플에서 검색에 최적화된 통합 텍스트 문자열을 생성합니다.
    머신러닝 기반의 의미론적 검색에 유리하도록 문맥을 구성합니다.

    Args:
        product_data_tuple (tuple): 상품 정보를 담고 있는 튜플.
                                  COLUMNS_INDICES에 정의된 순서와 일치해야 합니다.

    Returns:
        str: 검색 인덱싱을 위한 상품 정보를 결합한 문자열.
    """

    search_components = []

    # 튜플에서 데이터 추출 (COLUMN_INDICES를 활용하여 가독성을 높입니다)
    goods_nm = product_data_tuple[COLUMN_INDICES["GOODS_NM"]].strip()
    goods_stat_sct_nm = product_data_tuple[COLUMN_INDICES["GOODS_STAT_SCT_NM"]].strip()
    brnd_nm = product_data_tuple[COLUMN_INDICES["BRND_NM"]].strip()
    artc_info = product_data_tuple[COLUMN_INDICES["ARTC_INFO"]].strip()
    opt_disp_nm = product_data_tuple[COLUMN_INDICES["OPT_DISP_NM"]].strip()
    opt_val_desc = product_data_tuple[COLUMN_INDICES["OPT_VAL_DESC"]].strip()
    category_nms = product_data_tuple[COLUMN_INDICES["CATEGORY_NMS"]].strip()
    appliance_yn = product_data_tuple[COLUMN_INDICES["APPLIANCE_YN"]].strip()
    sch_kwd_nm = product_data_tuple[COLUMN_INDICES["SCH_KWD_NM"]].strip()
    
    # 숫자 값은 형 변환 시 에러 방지를 위해 get 대신 직접 접근 후 try-except 처리
    sale_prc_str = product_data_tuple[COLUMN_INDICES["SALE_PRC"]]
    dscnt_sale_prc_str = product_data_tuple[COLUMN_INDICES["DSCNT_SALE_PRC"]]
    card_dc_rate_str = product_data_tuple[COLUMN_INDICES["CARD_DC_RATE"]]
    max_benefit_price_str = product_data_tuple[COLUMN_INDICES["MAX_BENEFIT_PRICE"]]
    
    sale_prc = int(sale_prc_str) if sale_prc_str else None
    dscnt_sale_prc = int(dscnt_sale_prc_str) if dscnt_sale_prc_str else None
    card_dc_rate = int(card_dc_rate_str) if card_dc_rate_str else None
    max_benefit_price = int(max_benefit_price_str) if max_benefit_price_str else None

    card_dc_name = product_data_tuple[COLUMN_INDICES["CARD_DC_NAME"]].strip()


    # 1. 브랜드명 + 품목 정보 (카테고리/ARTC_INFO) + 상품명 조합
    # 상품의 핵심적인 정체성을 명확히 합니다.
    if brnd_nm and goods_nm:
        # ARTC_INFO에서 품목의 구체적인 타입을 추론하여 문맥에 더합니다.
        item_type = ''
        if artc_info:
            # '>' 기호로 구분된 품목 정보 중 가장 마지막 부분을 가져와 특정 품목 타입을 얻습니다.
            item_type_parts = [part.strip() for part in artc_info.split('>') if part.strip()]
            if item_type_parts:
                item_type = item_type_parts[-1]
                if '기타' in item_type: # '기타' 라는 모호한 품목이 포함되어 있는 경우
                    if category_nms: # 카테고리의 마지막 부분을 가져옵니다.
                        main_category_parts = [part.strip() for part in category_nms.split('>')]
                        if main_category_parts:
                            item_type = main_category_parts[0]


        # 품목 타입이 있고, 상품명에 이미 품목 타입이 포함되어 있지 않다면 추가합니다.
        if item_type and item_type not in goods_nm:
            search_components.append(f"{brnd_nm}의 {item_type}, {goods_nm}.")
        else: # 품목 타입이 없거나 상품명에 이미 포함되어 있다면 브랜드명과 상품명만 사용합니다.
            search_components.append(f"{brnd_nm}의 {goods_nm}.")
    elif goods_nm: # 브랜드명이 없는 경우 상품명만 사용합니다.
        search_components.append(f"{goods_nm}.")


    # 2. 제품의 주요 특성 (OPT_DISP_NM 및 OPT_VAL_DESC) 조합
    if opt_disp_nm and opt_val_desc:
        display_names = [name.strip() for name in opt_disp_nm.split(',') if name.strip()]
        values = [val.strip() for val in opt_val_desc.split(',') if val.strip()]

        # 각 특성 항목과 값을 '항목: 값' 형태로 결합합니다.
        characteristics = []
        for i in range(min(len(display_names), len(values))):
            # 비어있는 항목이나 '스마트기능'처럼 반복되는 용어는 건너뜁니다.
            # '스마트기능'이 반복되는 경우, 첫 번째 '스마트기능' 뒤에 모든 스마트기능 값을 나열합니다.
            if display_names[i] == '스마트기능':
                if i > 0 and display_names[i-1] == '스마트기능':
                    continue # 이미 처리되었으므로 건너뜁니다.
                
                # 모든 스마트기능 값을 모아서 처리
                all_smart_features = []
                for j in range(i, len(display_names)):
                    if display_names[j] == '스마트기능' and j < len(values):
                        all_smart_features.append(values[j])
                    else:
                        break # 스마트기능이 끝나면 반복 중단

                if all_smart_features:
                    characteristics.append(f"스마트기능: {', '.join(all_smart_features)}")
            else: # 일반적인 특성
                if display_names[i] and values[i]:
                    characteristics.append(f"{display_names[i]}: {values[i]}")

        if characteristics:
            search_components.append("주요 특징: " + ", ".join(characteristics) + ".")

    # 3. 카테고리명 추가
    if category_nms:
        # '|' 기호를 ', '로, '>' 기호를 ' > '로 바꾸어 카테고리 계층 및 다중 카테고리를 표현합니다.
        formatted_categories = category_nms.replace('|', ', ').replace('>', ' > ')
        search_components.append(f"카테고리: {formatted_categories}.")

    # 4. 상품 상태 추가
    if goods_stat_sct_nm and goods_stat_sct_nm != '정상상품': # '정상상품'이 아닌 경우에만 추가합니다.
        search_components.append(f"상품 상태: {goods_stat_sct_nm}.")

    # 5. 가격 및 할인 정보 추가
    price_info_components = []
    if sale_prc is not None:
        price_info_components.append(f"판매가 {sale_prc:,}원") # 천단위 콤마 추가
    if dscnt_sale_prc is not None and sale_prc is not None and dscnt_sale_prc < sale_prc:
        price_info_components.append(f"할인가 {dscnt_sale_prc:,}원") # 천단위 콤마 추가
    if card_dc_rate is not None and card_dc_rate > 0 and card_dc_name:
        if max_benefit_price is not None:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인되어 최대 {max_benefit_price:,}원") # 천단위 콤마 추가
        else:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인")

    if price_info_components:
        search_components.append("가격 정보: " + ", ".join(price_info_components) + ".")

    # 6. 가전제품 여부 추가
    if appliance_yn == 'Y':
        search_components.append("가전제품입니다.")
    else:
        search_components.append("가전제품이 아닙니다.")

    # 7. 해시태그 추가
    if sch_kwd_nm: 
        search_components.append(f"해시태그: {sch_kwd_nm}.")
    
    # 모든 구성 요소를 하나의 문자열로 결합하고 앞뒤 공백을 제거합니다.
    return " ".join(search_components).strip()

In [6]:
# 3. 상품정보를 문맥에 맞게 병합한 `search_text` 를 포함하여 documents 리스트 생성

documents = []
metadatas = []
ids = []

"""
- 예시: 삼성전자의 UHD TV, 125.7cm 4K UHD 비즈니스 TV LH50BEAHLGFXKR (설치유형 선택가능). 카테고리: TV·영상가전 > TV. 가격 정보: 판매가 559,000원.
"""
for row in rows:
    GOODS_NO, GOODS_NM, GOODS_STAT_SCT_NM, BRND_NM, ARTC_INFO, OPT_DISP_NM, OPT_VAL_DESC, CATEGORY_NMS, APPLIANCE_YN, SCH_KWD_NM, SALE_PRC, DSCNT_SALE_PRC, CARD_DC_RATE, CARD_DC_NAME, CARD_DC_NAME_LIST, MAX_BENEFIT_PRICE = row
    metadatas.append(
        {"GOODS_NO": GOODS_NO, 
         "GOODS_NM": GOODS_NM, 
         "GOODS_STAT_SCT_NM": GOODS_STAT_SCT_NM,
         "BRND_NM": BRND_NM,
         "ARTC_INFO": ARTC_INFO,
         "OPT_DISP_NM": OPT_DISP_NM,
         "OPT_VAL_DESC": OPT_VAL_DESC,
         "CATEGORY_NMS": CATEGORY_NMS,
         "APPLIANCE_YN": APPLIANCE_YN,
         "SCH_KWD_NM": SCH_KWD_NM,
         "SALE_PRC": int(SALE_PRC),
         "DSCNT_SALE_PRC": int(DSCNT_SALE_PRC),
         "CARD_DC_RATE": int(CARD_DC_RATE),
         "CARD_DC_NAME": CARD_DC_NAME,
         "CARD_DC_NAME_LIST": CARD_DC_NAME_LIST,
         "MAX_BENEFIT_PRICE": int(MAX_BENEFIT_PRICE)
         })
    search_text = generate_search_text_from_tuple(row)
    documents.append(search_text)
    ids.append(str(GOODS_NO))

# 샘플 출력
for i, doc in enumerate(documents[:10]):
    print(f"{i+1}. {doc}")


1. 엔산마운트의 TV·영상가전, 엔산 EL-09 이젤형 TV스탠드 티비거치대. 주요 특징: 색상: 화이트 계열. 카테고리: TV·영상가전 > TV > TV액세서리. 가격 정보: 판매가 176,000원. 가전제품이 아닙니다. 해시태그: #TV거치대#TV브라켓#모니터거치대#모니터브라켓.
2. 엔산마운트의 TV·영상가전, 엔산 무드조명형 모니터암 블룸스탠드 삼텐바이미 거치대. 카테고리: TV·영상가전 > TV > TV액세서리. 가격 정보: 판매가 140,000원. 가전제품이 아닙니다. 해시태그: #TV거치대#TV브라켓#모니터거치대#모니터브라켓.
3. 이스트라의 UHD TV, 5년무상AS 이스트라 120Hz AI 맥스 127cm UHD 구글 5.0 스마트TV (이젤스탠드A타입-기사). 주요 특징: 크기: 50인치(127cm), 종류: LED, 해상도: UHD(4K), 응답속도: 6m/s, 기본 주사율: 120Hz, 스마트기능: 구글플레이, 넷플릭스, 유튜브, 크롬캐스트, 구글어시스턴트, Disney+, 부가기능: 블루투스, 부가기능: 게임모드, 부가기능: 플리커프리, 부가기능: HDMI CEC, 부가기능: HDMI 2.1 지원, 부가기능: HDMI eARC, 부가기능: 크로마서브샘플링(4:4:4), 부가기능: USB재생(동영상·사진·음악), e효율등급: 1등급, 출시년도: 2025년, 화면 타입: 평면형, 스피커 출력: 24W, 음향효과: 돌비디지털Plus, 음향효과: 돌비디지털, 지원단자: HDMI. 카테고리: TV·영상가전 > TV > 중형(~139cm). 가격 정보: 판매가 588,000원. 가전제품입니다. 해시태그: #AN507UHD07.
4. 루컴즈의 LED TV, 2024년형 50인치 스마트 UHD 구글 TV 4K 에너지효율 1등급 T5003KUG 스탠드직배송. 주요 특징: 크기: 50인치(127cm), 종류: LED, 해상도: UHD(4K), 응답속도: 7m/s, 기본 주사율: 60Hz, 엔진: MT9612, 스마트기능: 넷플릭스, 유튜브, 크롬캐스트,

In [7]:
# 4. 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

한 번 요청에 임베딩 할 수 있는 토큰 수가 한정되어 있기 때문에 상품 개수를 잘라서 처리합니다.

In [8]:
# 5. DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.

# DB 경로 설정
persist_directory = "../.db/faiss"

# 배치 크기 설정
BATCH_SIZE = 500
total_products = len(documents)

# 첫 번째 배치로 FAISS 초기화
print(f"처리 중: 1~{min(BATCH_SIZE, total_products)}번째 상품 / 총 {total_products}개")
first_batch_docs = documents[:BATCH_SIZE]
first_batch_metadatas = metadatas[:BATCH_SIZE]
first_batch_ids = ids[:BATCH_SIZE]

vectorstore = FAISS.from_texts(
    texts=first_batch_docs,
    embedding=embeddings,
    metadatas=first_batch_metadatas,
    ids=first_batch_ids
)

# 나머지 배치 처리
for i in range(BATCH_SIZE, total_products, BATCH_SIZE):
    end_idx = min(i + BATCH_SIZE, total_products)
    print(f"처리 중: {i+1}~{end_idx}번째 상품 / 총 {total_products}개")
    
    batch_docs = documents[i:end_idx]
    batch_metadatas = metadatas[i:end_idx]
    batch_ids = ids[i:end_idx]
    
    try:
        vectorstore.add_texts(
            texts=batch_docs,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
    except Exception as e:
        print(f"배치 처리 중 오류 발생: {str(e)}")
        continue

# 로컬에 저장
vectorstore.save_local(persist_directory)

print(f"총 {total_products}개의 상품 정보가 FAISS에 저장되었습니다.")

처리 중: 1~500번째 상품 / 총 3726개
처리 중: 501~1000번째 상품 / 총 3726개
처리 중: 1001~1500번째 상품 / 총 3726개
처리 중: 1501~2000번째 상품 / 총 3726개
처리 중: 2001~2500번째 상품 / 총 3726개
처리 중: 2501~3000번째 상품 / 총 3726개
처리 중: 3001~3500번째 상품 / 총 3726개
처리 중: 3501~3726번째 상품 / 총 3726개
총 3726개의 상품 정보가 FAISS에 저장되었습니다.


임베딩 결과를 확인합니다.

In [9]:

# 6. 샘플 출력

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    k=1  # 반환할 결과 수
)

# 벡터스토어에서 임베딩된 데이터 확인
print("\n[벡터스토어 임베딩 데이터]")
print("-" * 60)
print(f"저장된 총 문서 수: {vectorstore.index.ntotal}")

# 문맥 샘플 확인
print(f"\n첫 번째 문서의 문맥 값:")
if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"{doc.page_content}")

# 임베딩 벡터 샘플 확인
sample_vector = vectorstore.index.reconstruct(0)  # 첫 번째 문서의 임베딩 벡터
print(f"\n첫 번째 문서의 임베딩 벡터 (처음 10개 값):")
print(sample_vector[:10])
print("-" * 60)


[벡터스토어 임베딩 데이터]
------------------------------------------------------------
저장된 총 문서 수: 3726

첫 번째 문서의 문맥 값:
MIDOTECH의 NLEDTV, LG 139cm 55NANO90 나노셀 4K 스마트 리퍼 TV 티비 (설치유형 선택가능). 카테고리: TV·영상가전 > TV > 중형(~139cm). 가격 정보: 판매가 910,000원. 가전제품입니다. 해시태그: #TV.

첫 번째 문서의 임베딩 벡터 (처음 10개 값):
[-0.00087856 -0.02148057 -0.08966528 -0.0302142  -0.01973385 -0.00383656
 -0.00490227  0.05464758  0.00777189 -0.02133501]
------------------------------------------------------------
